### Set Up

In [1]:
import findspark 
findspark.init() 

import pyspark
from delta import *
import pandas as pd
from pyspark.sql.functions import col

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
spark = pyspark.sql.SparkSession.builder.master("spark://spark-master:7077").appName("spark_kafka").config("spark.sql.streaming.metricsEnabled", "true").getOrCreate()

sc = spark.sparkContext
sc.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /opt/spark/cache
The jars for the packages stored in: /opt/spark/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.mysql#mysql-connector-j added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9c47bc50-feb4-44c9-8ce9-bf81cc0aae70;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.ap

23/03/12 19:57:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
import sys
sys.path.append('/opt/spark/work-dir/odp_intra_storage/ingestion_framework/')

### Bronze Metadata

In [ ]:
spark.sql("create database if not exists bronze").toPandas()

In [ ]:
spark.sql("show databases").toPandas()

In [ ]:
from metadata_refresh.bronze_refresh import refresh_bronze_metadata

In [ ]:
env = 'test'
bronze_metadata_file_location = '/opt/spark/work-dir/odp_intra_storage/vysh_temp/spark_clients/notebooks/bronze/bronze_metadata.csv'
bronze_metadata_table_name = 'bronze.metadata_table'

spark.sql(f"drop table if exists {bronze_metadata_table_name}")

refresh_bronze_metadata(spark, env, bronze_metadata_file_location, bronze_metadata_table_name)

In [ ]:
spark.table("bronze.metadata_table").where("pipeline_id = 1").select("*").toPandas()

### Streaming Pipeline

In [4]:
from spark_ingestion_app.spark_ingestion_app import SparkIngestionApp

In [5]:
# spark.sql("drop table if exists bronze.weather_test_v1").toPandas()

""


In [6]:
module_parameters = '\'{"group_id": "1", "pipeline_id": "1"}\''

SparkIngestionApp.run_module(spark = spark, module_name = 'KafkaToBronze', module_parameters = module_parameters, log_level='INFO')

In [7]:
spark.sql("select * from bronze.weather_test_v1 limit 10").toPandas()

/opt/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,key,topic,partition,offset,timestamp,timestampType,value,headers,kafka_date
0,None,weather_test_v1,0,860,2023-03-12 18:15:02.963,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
1,None,weather_test_v1,0,861,2023-03-12 18:15:33.813,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
2,None,weather_test_v1,0,862,2023-03-12 18:16:19.705,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
3,None,weather_test_v1,0,863,2023-03-12 18:17:20.502,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
4,None,weather_test_v1,0,864,2023-03-12 18:18:21.386,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
5,None,weather_test_v1,0,865,2023-03-12 18:19:22.144,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
6,None,weather_test_v1,0,866,2023-03-12 18:20:23.083,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116, 105, 109, 101, 34, 58, 32, 34, 83, 117, 110, 100, 97, 121, 32, 49, 58, 48, 48, 92, 117, 50, 48, 50, 102, 80, 77, 34, 44, 32, 34, 105, 110, 102, 111, 34, 58, 32, 34, 83, 110, 111, 119, 32, 115, 104, 111, 119, 101, 114, 115, 34, 44, 32, 34, 116, 101, 109, 112, 101, 114, 97, 116, ...]",None,2023-03-12
7,None,weather_test_v1,0,867,2023-03-12 18:21:23.838,0,"[123, 34, 100, 97, 116, 97, 34, 58, 32, 123, 34, 108, 111, 99, 97, 116, 105, 111, 110, 34, 58, 32, 34, 78, 97, 112, 101, 114, 118, 105, 108, 108, 101, 34, 44, 32, 34, 116,